# S2cloudless masking algorithm + grid download
https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
- This script uses the S2cloudless masking algorithm from the link above, and then creates a grid around my ROI so that I can download 55km cells into a local folder (55km cells maximizes GEE's download capability). I only export the masked image and calculated NDRE band. All other bands are excluded from download. Images exported at 20m scale.

In [1]:
import ee, geemap
print(ee.__version__)
print(geemap.__version__)

0.1.249
0.8.8


In [2]:
ee.Initialize()

In [3]:
admin1 = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .aggregate_array('ADM1_NAME')
     )
print(admin1.getInfo())

['Alagoas', 'Bahia', 'Ceara', 'Espirito Santo', 'Maranhao', 'Minas Gerais', 'Paraiba', 'Pernambuco', 'Piaui', 'Rio De Janeiro', 'Rio Grande Do Norte', 'Sao Paulo', 'Sergipe', 'Name Unknown', 'Name Unknown', 'Name Unknown', 'Acre', 'Amapa', 'Amazonas', 'Distrito Federal', 'Goias', 'Mato Grosso', 'Mato Grosso Do Sul', 'Para', 'Parana', 'Rio Grande Do Sul', 'Rondonia', 'Roraima', 'Santa Catarina', 'Tocantins', 'Name Unknown']


In [4]:
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
       .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
       .filterMetadata('ADM1_NAME', 'equals', 'Rondonia')
      )

In [5]:
Map = geemap.Map()
Map.addLayer(br, {}, 'Rondonia')
Map.center_object(br, zoom=6)
#Map

### Create polygon grid for clipping images

In [6]:
# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools

def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

In [7]:
# Make test grid (half degree squares)
grid_55km = make_grid(br, 55000)

In [8]:
# Access coordinates of grid squares
grid_dict = grid_55km.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)

In [9]:
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

In [10]:
# Make grid smaller if it's huge
idx = [64]
polys = [ polys[i] for i in idx]

In [11]:
# Make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys)

In [12]:
Map = geemap.Map()
Map.addLayer(grid, {}, 'grid')
Map.center_object(grid, zoom=7)
#Map

### Create advanced cloud mask

In [34]:
AOI = grid
START_DATE = '2016-01-01'
END_DATE = '2020-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [35]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    """Get Sentinel-2 data and join s2cloudless collection
    with Sentinel TOA data."""
    # Import and filter S2 TOA.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(aoi)
        .filterDate(ee.Date(start_date), ee.Date(end_date))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        .sort('CLOUDY_PIXEL_PERCENTAGE', False))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(ee.Date(start_date), ee.Date(end_date)))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    """Create and add cloud bands."""
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    """Create and add cloud shadow bands."""
    # Identify water pixels from the SCL band.
    #not_water = img.select('SCL').neq(6) <- this is for SR, not TOA
    not_water = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    not_water = not_water.select('NDWI')

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    """Use cloud shadow and shadow bands to create masks."""
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    """Mask cloudy pixels."""
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

### Add and Apply cloud mask

In [15]:
# Layers to display
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

In [16]:
# Run functions to create masks and new properties
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

# Object with cloud mask applied
s2_sr = (s2_sr_cld_col
         .map(add_cld_shdw_mask)
         .map(apply_cld_shdw_mask))

### Visualize components of advanced cloud mask

In [ ]:
# Create mosaic from collection
img = s2_sr_cld_col_eval_disp.mosaic()

# Subset layers and prepare for visualization
clouds = img.select('clouds').selfMask()
shadows = img.select('shadows').selfMask()
dark_pixels = img.select('dark_pixels').selfMask()
probability = img.select('probability')
cloudmask = img.select('cloudmask').selfMask()
cloud_transform = img.select('cloud_transform')

In [ ]:
# Create map for visualization
Map = geemap.Map()

Map.addLayer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'S2 Image')
Map.addLayer(clouds, {'palette': 'e056fd'}, 'clouds')
Map.addLayer(shadows, {'palette': 'yellow'}, 'shadows')
Map.addLayer(AOI, {}, 'AOI')

Map.center_object(AOI, zoom=7.5)
#Map

### Create NDRE band

In [17]:
# Calculate NDRE
def ndre_band(img):
    """Function to calculate ndre for an image."""
    ndre = img.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return img.addBands(ndre)

# Define NDRE palette
ndre_palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', 
           '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [18]:
# Apply NDRE function and select only the NDRE band for export
mskd_col = s2_sr.select('B5', 'B8')
ndre = mskd_col.map(ndre_band)
ndre_mskd_col = ndre.select('NDRE')

bands = ndre_mskd_col.first().bandNames()
print(bands.getInfo())

['NDRE']


### Clip NDRE to each polygon from grid

In [19]:
# Clip images in each collection to polygons
clipped_cols = []
for poly in polys:
    clipped_col = ndre_mskd_col.map(lambda image: image.clip(poly))
    clipped_cols.append(clipped_col)

print(clipped_cols)
print("\nTotal squares: " + str(len(clipped_cols)))
print(clipped_cols[0].first().bandNames().getInfo())


Total squares: 1
['NDRE']


### Visualize NDRE of one polygon

In [20]:
# Ensure images are clipped properly
Map = geemap.Map()
mean_ndre = clipped_cols[0].mean()

Map.addLayer(polys[0], {}, 'outline_1')
Map.addLayer(grid, {}, 'full_grid')
Map.addLayer(mean_ndre, {'palette':ndre_palette, 'min':0, 'max':1}, 'ndre')
Map.center_object(grid, zoom=10)

#Map

### Apply monthly step to data before export

In [55]:
#Monthly step
#https://gis.stackexchange.com/questions/301165/how-to-get-monthly-averages-from-earth-engine-in-the-python-api
months = ee.List.sequence(1, 12) #Create a list with nums 1-12
years = ee.List.sequence(2017, 2020)

all_cols = []
for a_col in clipped_cols:
    def byYear(y):
        def byMonth(m):
            return (a_col
                    .filter(ee.Filter.calendarRange(y, y, 'year'))
                    .filter(ee.Filter.calendarRange(m, m, 'month'))
                    .mean()
                    .set('month', m)
                    .set('year', y)
                   )
        return months.map(byMonth)
    
    col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
    all_cols.append(col)

column = all_cols[0].filter(ee.Filter.eq('system:index', '1'))
print(column.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'NDRE', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'month': 2, 'year': 2017, 'system:index': '1'}}]}


### Optionally apply 15-day step

In [1]:
#15-day step
years = ee.List.sequence(2016, 2019)
step = ee.List.sequence(1, 365, 15)

all_cols = []
for a_col in clipped_cols:
    def byYear(y):
        y = ee.Number(y)
        def byStep(d):
            d = ee.Number(d)
            return (a_col
                    #.filter(ee.Filter.calendarRange(y, y, 'year')) #yearly step
                    #.filter(ee.Filter.calendarRange(d, d.add(14), 'day_of_year')) #15-day step
                    .mean()
                    .set('step', [d, y])) #Add properties
                
        return step.map(byStep)

    col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
    all_cols.append(col)
    
column = all_cols[0].filter(ee.Filter.eq('system:index', '1'))
print(column.getInfo())

NameError: name 'ee' is not defined

### Visualize NDRE time series

In [26]:
import matplotlib.pyplot as plt
import numpy as np

In [27]:
# Mean
def region_mean_Image(img):
    # mean over all pixels in the region for the image
    return img.set('mean', img.reduceRegion(ee.Reducer.mean(), geometry=polys[0], scale=30)) 

def region_mean_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_mean_Image)
    stats_list = stats.aggregate_array('mean')
    return np.array(stats_list.getInfo())


# Median
def region_median_Image(img):
    # mean over all pixels in the region for the image
    return img.set('median', img.reduceRegion(ee.Reducer.median(), geometry=polys[0], scale=30)) 

def region_median_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_median_Image)
    stats_list = stats.aggregate_array('median')
    return np.array(stats_list.getInfo())

In [28]:
# Make mean and median lists from collection
mean_list = region_mean_ImageCollection(all_cols[0])
median_list = region_median_ImageCollection(all_cols[0])

KeyboardInterrupt: 

In [ ]:
print(mean_list)

In [ ]:
# Make list of values
empty = {}
def newList(alist):
    list_name = []
    for value in alist:
        if value != empty:
            list_name.append(value)
        else:
            list_name.append({'NDRE': np.nan})
    return list_name

In [ ]:
# Make mean and median value lists
mean_ndre = newList(mean_list)
median_ndre = newList(median_list)

In [ ]:
print(mean_ndre)

In [ ]:
def valList(aList):
    new_list = []
    for d in aList:
        for key, val in d.items():
            new_list.append(val)
    return new_list

In [ ]:
a_mean = np.array(valList(mean_ndre))
a_median = np.array(valList(median_ndre))
print(a_mean)

In [ ]:
# Create month list
months = np.array(range(len(a_mean)))
print(months)

In [ ]:
# Make plot
x = months
y = a_mean
m = a_median

#ticks1 = list(range(0,48))
plt.axes([0, 0, 2, 1])
plt.grid(b = True, which = 'major', axis = 'x')
plt.title('2017-2020 mean and median NDRE')
#plt.fill_between(x, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")

mean = plt.plot(x,y, color = "black", label = "Mean")
median = plt.plot(x,m, color = "red", label = "Median")

plt.legend(loc = "lower right")
#plt.xticks(ticks = ticks1)
#plt.xticks(ticks = [0, 6, 12, 18, 24, 30, 36, 42, 47], labels = ['2016', 'Jul', '2017', 'Jul', '2018', 'Jul', '2019', 'Jul', 'Dec'])
plt.show()

### Export monthly images from each collection

In [22]:
size = all_cols[0].size()
print(size.getInfo())

48


In [56]:
# Make a list of file names
tiles = []
sitename = 'rondonia'
for num in range(len(all_cols)):
    index = str(sitename + '_{}'.format(num))
    tiles.append(index)

print(tiles)

['rondonia_0']


In [57]:
import os
import time

In [58]:
# Export monthly images from a collection
tic1 = time.time()
for a_col, a_tile, poly in zip(all_cols, tiles, polys):
    ilist = a_col.toList(a_col.size())
    for i in range(12*4):
        if len(ee.Image(ilist.get(i)).bandNames().getInfo()) <= 0:
            print("ERROR; No bands found in image index %d... will skip export."%(i))
        else:
            filename = "/data/6ru/sentinel_2/rondonia/{}/{}.tif".format(a_tile,i)
            temp_dir = "/data/6ru/sentinel_2/rondonia/{}/".format(a_tile)
            if not os.path.exists(temp_dir):
                os.mkdir(temp_dir)
            if os.path.exists(filename):
                print("Image {} already exists. Skipping...".format(i))
                next
            else:
                tic = time.time()
                print("Exporting Image %d"%(i))
                geemap.ee_export_image(ee.Image(ilist.get(i)).select('NDRE'), 
                                       filename=filename, 
                                       scale=30, 
                                       region=poly, 
                                       file_per_band=False)
                toc = time.time()
                hours, rem = divmod(toc-tic, 3600)
                mins, secs = divmod(rem, 60)
                print("Time elapsed: {:0>2}:{:0>2}:{:05.2f}"
                      .format(int(hours),int(mins),secs))
toc1 = time.time()
hrs1, rem1 = divmod(toc1-tic1, 3600)
mins1, secs1 = divmod(rem1,  60)
print("Total time elapsed: {:0>2}:{:0>2}:{:05.2f}"
      .format(int(hrs1),int(mins1),secs1))

ERROR; No bands found in image index 0... will skip export.
Exporting Image 1
Generating URL ...
Please wait ...
Data downloaded to /data/6ru/sentinel_2/rondonia/rondonia_0/1.tif
Time elapsed: 00:00:58.29
ERROR; No bands found in image index 2... will skip export.
Exporting Image 3
Generating URL ...
Please wait ...
Data downloaded to /data/6ru/sentinel_2/rondonia/rondonia_0/3.tif
Time elapsed: 00:01:07.32
Exporting Image 4
Generating URL ...
Please wait ...


KeyboardInterrupt: 